In [0]:
#@title <b><font color="red" size="+1">←</font><font color="black" size="+1"> prepare supplementary materials for the lesson</font></b>
# prepare supplementary materials for the lesson
!rm -rf harbour-space-text-mining-course
!git clone https://github.com/horoshenkih/harbour-space-text-mining-course.git
import sys
sys.path.append('harbour-space-text-mining-course')

from collections import Counter
from math import exp
from tabulate import tabulate
from tqdm import tqdm_notebook
from IPython.display import HTML, display

Cloning into 'harbour-space-text-mining-course'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 70 (delta 20), reused 57 (delta 10), pack-reused 0
Unpacking objects: 100% (70/70), done.


# Lesson 1

В этом уроке мы будем исследовать [архив твитов Трампа](http://trumptwitterarchive.com/archive), который хранится в репозитории курса в виде json-файла.

In [0]:
import json
with open("harbour-space-text-mining-course/datasets/trump_twitter_archive/tweets.json") as f:
  tweets = json.load(f)

Твиты хранятся в виде списка словарей:

In [0]:
from pprint import pprint
print("type:", type(tweets))
print("num tweets:", len(tweets))
pprint(tweets[0])

type: <class 'list'>
num tweets: 45868
{'created_at': 'Sat Feb 22 04:17:45 +0000 2020',
 'favorite_count': 47513,
 'id_str': '1231070547607511040',
 'is_retweet': False,
 'retweet_count': 11327,
 'source': 'Twitter for iPhone',
 'text': 'Incredible people, great Rally! https://t.co/3i6tgfqrRl'}


В этом уроке нам будут интересны тексты твитов и время их создания.

# Tokenization
Как мы читаем письменный текст?
Технически, текст -- это последовательность букв, но мы автоматически воспринимаем его как последовательность **токенов** -- слов, знаков препинания и прочих подстрок, несущих смысловую нагрузку.

Простейший способ токенизации -- разбить по whitespace (это делает метод строк `.split()`):


In [0]:
text = tweets[0]["text"]
print("Text:")
pprint(text)

Text:
'Incredible people, great Rally! https://t.co/3i6tgfqrRl'


In [0]:
pprint(text.split(), compact=True)
print()

['Incredible', 'people,', 'great', 'Rally!', 'https://t.co/3i6tgfqrRl']



Видно, что нужно отделить знаки препинания от слов, за которыми они следуют, при этом корректно обработав случаи, когда знак препинания является частью слова, url или логина Твиттера:

In [0]:
text_to_tokenize = None
for i, tweet in enumerate(tweets):
  special_chars = "-'@/"
  tweet_chars = set(tweet["text"])
  if all([c in tweet_chars for c in special_chars]):
    text_to_tokenize = tweet["text"]
    print(i)
    pprint(tweet["text"])
    break

537
("RT @nypost: Nancy Pelosi 'pre-ripped' pages of Trump's SOTU speech, video "
 'shows https://t.co/e4bItGSTPt https://t.co/Ef8YeIVGCx')


Не стоит решать задачу токенизации самостоятельно.
Пришло время познакомиться с spaCy -- библиотекой для обработки естественного текста

In [0]:
# text to be tokenized
pprint(text_to_tokenize)

from spacy.lang.en import English

# create text analyzer that includes rules of tokenization
analyze_text = English()

# spaCy has functional API
analyzed_text = analyze_text(text_to_tokenize)

# the resulting object is iterable, and tokens can be extracted by iteration
pprint([token.text for token in analyzed_text], compact=True)

("RT @nypost: Nancy Pelosi 'pre-ripped' pages of Trump's SOTU speech, video "
 'shows https://t.co/e4bItGSTPt https://t.co/Ef8YeIVGCx')
['RT', '@nypost', ':', 'Nancy', 'Pelosi', "'", 'pre', '-', 'ripped', "'",
 'pages', 'of', 'Trump', "'s", 'SOTU', 'speech', ',', 'video', 'shows',
 'https://t.co/e4bItGSTPt', 'https://t.co/Ef8YeIVGCx']


Let's find the most frequent tokens

In [0]:
from spacy.lang.en import English
from tqdm import tqdm_notebook  # show progress bar

nlp = English()
token_counter = Counter()
for tweet in tqdm_notebook(tweets[:1000]):
  doc = nlp(tweet["text"])
  for token in doc:
    token_counter[token.text] += 1
pprint(token_counter.most_common()[:20])


[(',', 960),
 ('the', 890),
 ('.', 760),
 (':', 709),
 ('RT', 590),
 ('to', 556),
 ('!', 435),
 ('…', 426),
 ('and', 387),
 ('of', 352),
 ('a', 327),
 ('is', 308),
 ('in', 289),
 ('for', 203),
 ('I', 161),
 ('that', 159),
 ('#', 159),
 ('@realDonaldTrump', 155),
 ('\n\n', 143),
 ('on', 142)]


Note that tokens "The" and "the" are treated as different words, but have the same meaning (there is also a token "THE").
So it makes sense to convert most of the tokens to lowercase.
It is called *normalization*.
However, some tokens should preserve uppercase (for example, the `Trump` token should be in uppercase).

# Lemmatization

Иногда важно не только приводить слова к одному регистру, но и приводить разные формы слов к одной, *канонической* форме (грубо говоря, к той форме, по которой это слово можно найти в словаре).
Эта форма называется *леммой*.
Посмотрим, как получить леммы слов в spaCy.


In [0]:
import spacy
nlp = spacy.load("en")  # load dictionaries for lemmatization
for token in nlp("Don't mess with Donald Trump."):
  print("'{}' -> '{}'".format(token, token.lemma_))  # `lemma_` attribute contains the text of the token's lemma

'Do' -> 'do'
'n't' -> 'not'
'mess' -> 'mess'
'with' -> 'with'
'Donald' -> 'Donald'
'Trump' -> 'Trump'
'.' -> '.'


Заметим, что `Don't` превращается в `do not`, т.е.
- приводится к нижнему регистру;
- сокращенная форма `n't` превращается в полную `not`.

Также `Donald` и `Trump` не приводятся к нижнему регистру.
А вот `Trump` (без предшествующего `Donald`) -- приводится.

In [0]:
for token in nlp("Trump is a noun."):
  print("'{}' -> '{}'".format(token, token.lemma_))  # `lemma_` attribute contains the text of the token's lemma

'Trump' -> 'Trump'
'is' -> 'be'
'a' -> 'a'
'noun' -> 'noun'
'.' -> '.'


Как это работает? Под капотом spaCy определяет лемму в том числе на основе предсказанной части речи для этого слова. Именно "предсказанной", потому что в общем случае нельзя определить часть речи только по написанию.

Часть речи токена можно получить с помощью атрибута `tag_`.

In [0]:
for text in ("Trump is a noun.", "Donald Trump."):
  for token in nlp(text):
    print("'{}' -> '{}'".format(token, token.tag_))  # `tag_` attribute contains the text of the token's predicted part-of-speech tag

'Trump' -> 'NNP'
'is' -> 'VBZ'
'a' -> 'DT'
'noun' -> 'NN'
'.' -> '.'
'Donald' -> 'NNP'
'Trump' -> 'NNP'
'.' -> '.'


To make `tag_` human-readable, use `spacy.explain` function:

In [0]:
for text in ("Trump is a noun.", "Donald Trump."):
  for token in nlp(text):
    print("'{}' -> '{}' ({})".format(token, token.tag_, spacy.explain(token.tag_)))  # `spacy.explain` makes tag_ human-readable

'Trump' -> 'NNP' (noun, proper singular)
'is' -> 'VBZ' (verb, 3rd person singular present)
'a' -> 'DT' (determiner)
'noun' -> 'NN' (noun, singular or mass)
'.' -> '.' (punctuation mark, sentence closer)
'Donald' -> 'NNP' (noun, proper singular)
'Trump' -> 'NNP' (noun, proper singular)
'.' -> '.' (punctuation mark, sentence closer)


# NLP pipeline

Кроме частей речи, `spacy` по умолчанию извлекает и другую информацию из текстов. Последовательность шагов по извлечению этой информации объединяется в так называемый _pipeline_.

![NLP pipeline](https://spacy.io/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg)

Подробнее: https://spacy.io/usage/processing-pipelines

In [0]:
# `nlp` object has the attribute `pipeline`
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7f4e6c16d080>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7f4e6c166b88>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7f4e6c166be8>)]

`tagger` извлекает части речи.
Давайте кратко ознакомимся с `parser` и `ner`.

## parser
Компонент `parser` (по умолчанию это объект класса `spacy.pipeline.pipes.DependencyParser`) извлекает зависимости между словами.
Посмотрим на примере:

In [0]:
from spacy import displacy

doc = nlp("This is a sentence.")
html = displacy.render(doc, style="dep")
HTML(html)

## ner
Компонент `ner` (что означает "Named Entity Recognition") распознает именованные сущности, которые соответствуют объектам реального мира.

Подробнее: https://spacy.io/usage/linguistic-features#named-entities

Рассмотрим примеры таких сущностей в твитах Трампа:

In [0]:
from spacy import displacy

for tweet in tweets:
  doc = nlp(tweet["text"])
  entities = list(doc.ents)  # the `ents` attribute contains a sequence of found named entities (if any)
  if entities:
    break

for ent in entities:
  entity_text = ent.text
  entity_label = ent.label_  # each entity is assigned a label, stored in the `label_` attribute
  explained_entity_label = spacy.explain(ent.label_)  # spacy.explain() works with entity labels

  print("Entity text: {}\nEntity label: {} ({})\n".format(entity_text, entity_label, explained_entity_label))
html = displacy.render(doc, style="ent")
HTML(html)

Entity text: Russia
Entity label: GPE (Countries, cities, states)



## Customizing pipelines
Как мы видим, по умолчанию spaCy включает в себя много компонент для анализа текста.
Однако, каждая компонента вносит некоторую задержку в скорость обработки текстов.
Поэтому рекомендуется оставлять только нужные компоненты.

In [0]:
nlp_all = spacy.load("en")  # load tagger, parser, ner
nlp_tagger = spacy.load("en", disable=["parser", "ner"])  # load only tagger
nlp_none = spacy.load("en", disable=["parser", "ner", "tagger"])  # disable everything

In [0]:
%%timeit
for tweet in tweets[:100]:
  nlp_all(tweet["text"])

1 loop, best of 3: 1.06 s per loop


In [0]:
%%timeit
for tweet in tweets[:100]:
  nlp_tagger(tweet["text"])

1 loop, best of 3: 290 ms per loop


In [0]:
%%timeit
for tweet in tweets[:100]:
  nlp_none(tweet["text"])

The slowest run took 32.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.7 ms per loop


# "Слово дня"

Ппоробуем решить такую задачу: для каждого дня найти слово, "характеризующее" этот день.
Например, это могло бы быть название какого-нибудь события, страны или имя человека.

Сперва попробуем найти за каждый день найти слово, которое упоминалось в этот день чаще всего.

In [0]:
from dateutil.parser import parse as parse_datetime  # convert a string in any format to a datetime object
from collections import Counter, defaultdict

nlp = spacy.load("en", disable=["parser", "ner", "tagger"])
date_word_count = defaultdict(Counter)  # count words for each date
for tweet in tqdm_notebook(tweets[:10000]):
  tweet_date = parse_datetime(tweet["created_at"]).date()
  tweet_text = tweet["text"]

  for token in nlp(tweet_text):
    date_word_count[tweet_date][token.lemma_] += 1  # here we count lemmas

In [0]:
# visualize
from tmcourse.utils import calendar_table  # supplementary code for the course

times = []
values = []
weights = []

for d in sorted(date_word_count):
  # Counter.most_common(n) returns a list of n most frequent pairs (element, frequency)
  times.append(d)
  word, count = date_word_count[d].most_common(1)[0]
  values.append(word)
  weights.append(count)

html = calendar_table(times, values, weights)
display(HTML(html))

week,Mon,Tue,Wed,Thu,Fri,Sat,Sun
2018-11-05 - 2018-11-11,,,Attorney,.,to,",",the
2018-11-12 - 2018-11-18,",",the,",",the,the,be,be
2018-11-19 - 2018-11-25,be,the,.,",",.,.,.
2018-11-26 - 2018-12-02,",",",",be,",",.,and,the
2018-12-03 - 2018-12-09,.,.,be,",",the,be,not
2018-12-10 - 2018-12-16,be,.,and,",",be,",",the
2018-12-17 - 2018-12-23,",",",",",",",",the,be,","
2018-12-24 - 2018-12-30,be,be,.@FLOTUS,the,the,the,.
2018-12-31 - 2019-01-06,be,be,.,and,",",",",.
2019-01-07 - 2019-01-13,the,the,",",the,",",",",the


Ожидаемо, самыми частотными леммами оказались местоимения, артикли и знаки препинания.
Но, несмотря на частотность, они не являются информативными.
Вопрос: как найти информативные слова?

# TF-IDF

Для начала, необходимо уточнить, какие слова являются информативными.
Мы видели. что самые частые слова не являются самыми информативными.
Нетрудно видеть, что самые редкие слова тоже не несут информации.

In [0]:
# visualize
times = []
values = []
weights = []

for d in sorted(date_word_count):
  # Counter.most_common(n) returns a list of n (all if n == None) most frequent pairs (element, frequency)
  times.append(d)
  word, count = date_word_count[d].most_common()[-1]
  values.append(word)
  weights.append(count)

html = calendar_table(times, values, weights)
display(HTML(html))

week,Mon,Tue,Wed,Thu,Fri,Sat,Sun
2018-11-05 - 2018-11-11,,,Country,bar,#,...,Smart
2018-11-12 - 2018-11-18,https://t.co/JPUkOr4rW1,affect,worker,dishonest,Freedom...https://t.co/TuEy635mav,give,https://t.co/B1MCTF83Zf
2018-11-19 - 2018-11-25,Enjoy,Associat,?,?,NOW,…,@marklevinshow
2018-11-26 - 2018-12-02,separate,-@realDonaldTrum,Subpoena,learn,job,https://t.co/4IHvUdOygc,https://t.co/Kdjeyvkzmb
2018-12-03 - 2018-12-09,%,https://t.co/9xbFYlZzNs,....,@LouDobbs,Win,…,Steele
2018-12-10 - 2018-12-16,soon,start,Border,WALL,their,job,belittle
2018-12-17 - 2018-12-23,American,protest,fantastic,…,Now,recovery,event
2018-12-24 - 2018-12-30,…,…,https://t.co/rDlhITDvm1,how,approve,Demanded,with
2018-12-31 - 2019-01-06,sense,thrive,Bad,government,overdue,@LouDobbs,@DRUDGE_REPORT
2019-01-07 - 2019-01-13,TRUTH,end,https://t.co/Ft6FqQmYfI,possible,https://t.co/NAuMaQW6fl,sleaze,within


Попробуем такую идею: слово считаем информативным, если оно "в среднем" встречается относительно редко, но в конкретный день встречается относительно часто.

Формализуем эту идею на языке вероятностей.

Пусть имеется коллекция документов $D$, и для каждого документа $d \in D$ мы хотим узнать, какие слова лучше всего характеризуют этот документ.
Пусть словарь конечен и содержит $N$ слов.
Пусть $p_i$ -- вероятность встретить слово $i$ в случайном документе из коллекции $D$, а $t_i$ -- сколько раз слово $i$ встретилось в конкретном документе $d $ ($t_i$ может быть равно 0 -- это означает, что $i$-е слово ни разу не встретилось в документе $d$).
Тогда произведение $\prod_{i=1}^{N} p_i^{t_i}$ даёт вероятность того, что этот документ получится "случайно".
Обратная величина $L_d = \prod_{i=1}^{N} p_i^{-t_i}$ как бы характеризует, насколько документ "не случаен". Возьмём логарифм:
$$
\log L_d = \sum_{i=1}^{N}t_i\log\left(\frac{1}{p_i}\right)
$$
Чем больше $i$-е слагаемое, тем сильнее $i$ слово делает документ "неслучайным".

Каждое слагаемое состоит из двух множителей:
- **Term frequency** $t_i$ -- сколько раз слово встретилось в документе. Чем чаще, тем "характернее" слово для данного документа
- **Inversed document frequency** $\log\left(\frac{1}{p_i}\right)$. Чем больше эта величина, тем реже слово $i$ встречается в _других_ документах.



In [0]:
from math import log

# visualize
times = []
values = []
weights = []

# IDF
word_counter = Counter()
num_documents = len(date_word_count)
for words in date_word_count.values():
  for word in words:
    word_counter[word] += 1

for d in sorted(date_word_count):
  times.append(d)
  word_tfidf = {}
  for word, count in date_word_count[d].items():
    tf = count
    idf = log(num_documents / word_counter[word])
    word_tfidf[word] = tf * idf

  word, tfidf = list(sorted(word_tfidf.items(), key=lambda x: x[1]))[-1]  # get the highest TF-IDF value
  values.append(word)
  weights.append(tfidf)

html = calendar_table(times, values, weights)
display(HTML(html))

week,Mon,Tue,Wed,Thu,Fri,Sat,Sun
2018-11-05 - 2018-11-11,,,Sessions,scene,Broward,Whitaker,Suresnes
2018-11-12 - 2018-11-18,ballot,France,w/,Veterans,Midterm,Governor,Wallace
2018-11-19 - 2018-11-25,Bin,Christmas,9th,Thanksgiving,major,Small,Foundation
2018-11-26 - 2018-12-02,child,Motors,@The_Trump_Train,book,Argentina,H.W.,Hanukkah
2018-12-03 - 2018-12-09,China,clean,deadly,Doug,Report,Paris,Opened
2018-12-10 - 2018-12-16,Smocking,Comey,9:00pmE,liability,Hikes,Interior,Required
2018-12-17 - 2018-12-23,Anytime,Isikoff,Foundation,ISIS,217,https://t.co/ZGcYygMf3a,Corker
2018-12-24 - 2018-12-30,Wall,Christmas,https://t.co/rDlhITDvm1,DACA,Honduras,Texts,compound
2018-12-31 - 2019-01-06,yes,ALWAYS,Mitt,https://t.co/vtqnUwdhjB,Defense,Abramson,plus
2019-01-07 - 2019-01-13,knowingly,Industry,unlikely,@charliekirk11,station,Shutdown,SPECIAL


# TF-IDF in sklearn

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = [
  "One, two, three.",
  "Two, three.",
  "Three."
]

# idf = log(n_documents / count) + 1
# smooth_idf = log((n_documents + 1) / (count + 1)) + 1
vectorizer = TfidfVectorizer(smooth_idf=False)
vectorizer.fit(data)

vectorizer_internals = []

# vectorizer.vocabulary_ contains word -> index mapping
# vectorizer.idf_ contains the idf value for each index
for word, idx in vectorizer.vocabulary_.items():
  idf = vectorizer.idf_[idx]
  count = len(data) / exp(idf - 1)  # here we restore word count from idf
  vectorizer_internals.append([word, idf, count])

print(tabulate(vectorizer_internals, headers=["word", "idf", "count"]))

word        idf    count
------  -------  -------
one     2.09861        1
two     1.40547        2
three   1              3


Обученный векторизатор можно применять на новых текстах.

In [0]:
pprint(vectorizer.transform(["Two, three, four", "Five"]).todense())

matrix([[0.        , 0.57973867, 0.81480247],
        [0.        , 0.        , 0.        ]])


Метод `transform()` преобразует массив входных текстов в матрицу, где строка соответствует тексту, а $i$-й столбец соответствует слову из обучающего множества, и это соответствие хранится в атрибуте `vocabulary_`.

Заметим, что:
1. Новые слова ("four", "five") игнорируются.
1. Ненулевые векторы нормализованы (имеют длину 1).

По умолчанию `TfidfVectorizer` делает минимальную предобработку текста, в частности приведение к нижнему регистру и разбиение на токены по знакам препинания.

In [0]:
text = tweets[0]["text"]
print(text)
vectorizer = TfidfVectorizer().fit([text])
pprint(vectorizer.vocabulary_)

Incredible people, great Rally! https://t.co/3i6tgfqrRl
{'3i6tgfqrrl': 0,
 'co': 1,
 'great': 2,
 'https': 3,
 'incredible': 4,
 'people': 5,
 'rally': 6}


## sklearn + spaCy
Но метод обработки текста можно переопределить.

In [0]:
nlp = spacy.load("en", disable=["parser", "ner", "tagger"])

def spacy_tokenizer(text):
  return [t.lemma_ for t in nlp(text)]

text = tweets[0]["text"]
print(text)
vectorizer_spacy = TfidfVectorizer(tokenizer=spacy_tokenizer).fit([text])
pprint(vectorizer_spacy.vocabulary_)

Incredible people, great Rally! https://t.co/3i6tgfqrRl
{'!': 0,
 ',': 1,
 'great': 2,
 'https://t.co/3i6tgfqrrl': 3,
 'incredible': 4,
 'people': 5,
 'rally': 6}


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


# Text similarity via TF-IDF: Quora Question Pairs

In [0]:
import pandas as pd

df = pd.read_csv("harbour-space-text-mining-course/datasets/quora_question_pairs/train.csv")
df = df.head(20000)  # select a subset of rows to speed up the demonstration
df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df.fillna("."), test_size=0.5, shuffle=True, random_state=0)
quora_vectorizer = TfidfVectorizer(tokenizer=spacy_tokenizer)
questions = np.hstack([df_train.question1.values, df_train.question2.values])

In [0]:
quora_vectorizer.fit(questions)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function spacy_tokenizer at 0x7f4e689a87b8>,
                use_idf=True, vocabulary=None)

In [0]:
# extract TF-IDF vectors
question_1_vectors = quora_vectorizer.transform(df_test.question1)
question_2_vectors = quora_vectorizer.transform(df_test.question2)

In [0]:
from sklearn.metrics import roc_auc_score

# compute distances between TF-IDF vectors
# iterate over pairs using `zip()` generator
tf_idf_similarity = [
  # v1, v2 are sparse matrices: convert their dot product into a dense matrix and then to a scalar
  np.dot(v1, v2.T).todense().item()  
  for v1, v2 in tqdm_notebook(zip(question_1_vectors, question_2_vectors), total=question_1_vectors.shape[0])
]
# evaluate ROC AUC score
print("ROC AUC:", roc_auc_score(df_test.is_duplicate, tf_idf_similarity))


ROC AUC: 0.7190145081862447


# [OPTIONAL] sklearn vectorizers

# [OPTIONAL] sklearn pipelines

# Ссылки
- [Natural Language Processing with Python, chapter 3
](http://www.nltk.org/book/ch03.html)
- [Advanced NLP with spaCy, chapter 1](https://course.spacy.io/)
- https://en.wikipedia.org/wiki/Lemma_(morphology)